In [ ]:
# Autoregressive Moving Average Model (1,1)

code = """
data {
  int<lower=1> T;          // num observations
  real y[T];               // observed outputs
}
parameters {
  real mu;                 // mean coeff
  real phi;                // autoregression coeff
  real theta;              // moving avg coeff
  real<lower=0> sigma;     // noise scale
}
model {
  vector[T] nu;           // prediction for time t
  vector[T] err;          // error for time t
  nu[1] = mu + phi * mu;  // assume err[0] == 0
  err[1] = y[1] - nu[1];
  for (t in 2:T) {
    nu[t] = mu + phi * y[t-1] + theta * err[t-1];
    err[t] = y[t] - nu[t];
  }
  mu ~ normal(0, 10);         // priors
  phi ~ normal(0, 2);
  theta ~ normal(0, 2);
  sigma ~ cauchy(0, 5);
  err ~ normal(0, sigma);      // likelihood
}
"""
data = {
    'T': len (player),
    'y': player.tolist()
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=1)
print fit
fit.plot(['mu','phi', 'sigma', 'theta'])
plt.tight_layout()

In [ ]:
// Moving Average MA(2)

code = """
data {
      int<lower=3> T;  // number of observations
      vector[T] y;     // observation at time T
    }
    parameters {
      real mu;              // mean
      real<lower=0> sigma;  // error scale
      vector[2] theta;      // lag coefficients
    }
    transformed parameters {
      vector[T] epsilon;    // error terms
      epsilon[1] = y[1] - mu;
      epsilon[2] = y[2] - mu - theta[1] * epsilon[1];
      for (t in 3:T)
        epsilon[t] = ( y[t] - mu
                        - theta[1] * epsilon[t - 1]
                        - theta[2] * epsilon[t - 2] );
} model {
      mu ~ cauchy(0, 2.5);
      theta ~ cauchy(0, 2.5);
      sigma ~ cauchy(0, 2.5);
      for (t in 3:T)
        y[t] ~ normal(mu
                      + theta[1] * epsilon[t - 1]
                      + theta[2] * epsilon[t - 2],
                      sigma);
}
"""
data = {
    'T': len (player),
    'y': player.tolist()
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=1)
print fit
fit.plot(['mu','sigma', 'theta'])
plt.tight_layout()